In [ ]:
!pip install imblearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTENC
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTENC
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

from scipy.optimize import linear_sum_assignment as linear_assignment

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'{device} is available')

cuda:0 is available


## 대전 Cluster 345

In [3]:
chungnam_345 = pd.read_csv('/content/drive/MyDrive/지역 치안 안전 데이터 분석/data/modeling data/chungnam_345.csv')
display(chungnam_345)
chungnam_345.isnull().sum()

,date,accident_num,평균기온(°C),일강수량(mm),평균 풍속(m/s),평균 상대습도(%),평균 이슬점온도(°C),year,month,day,측정소명,O3,PM10,PM2.5,season,요일,workday
0,2020-01-01,0.0,0.3,0.0,0.7,68.1,-5.0,2020,1,1,성황동,0,1,1,winter,수요일,1
1,2020-01-02,0.0,1.8,0.0,0.8,73.4,-2.5,2020,1,2,성황동,0,1,2,winter,목요일,0
2,2020-01-03,0.0,2.5,0.0,0.6,73.8,-1.9,2020,1,3,성황동,0,2,2,winter,금요일,0
3,2020-01-04,0.0,1.8,0.0,0.9,76.0,-2.1,2020,1,4,성황동,0,1,2,winter,토요일,1
4,2020-01-05,0.0,2.0,0.0,1.3,54.9,-6.8,2020,1,5,성황동,0,1,2,winter,일요일,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1122,2023-01-27,0.0,-8.0,0.0,2.0,63.0,-14.6,2023,1,27,성황동,0,1,1,winter,금요일,0
1123,2023-01-28,0.0,-5.7,0.0,1.5,63.1,-12.1,2023,1,28,성황동,0,1,0,winter,토요일,1
1124,2023-01-29,0.0,-1.9,0.0,1.5,70.4,-6.9,2023,1,29,성황동,0,1,0,winter,일요일,1
1125,2023-01-30,0.0,-1.6,0.0,1.3,58.3,-9.8,2023,1,30,성황동,0,1,0,winter,월요일,0


date            0
accident_num    0
평균기온(°C)        0
일강수량(mm)        0
평균 풍속(m/s)      0
평균 상대습도(%)      0
평균 이슬점온도(°C)    0
year            0
month           0
day             0
측정소명            0
O3              0
PM10            0
PM2.5           0
season          0
요일              0
workday         0
dtype: int64

In [4]:
chungnam_345['date'] = pd.to_datetime(chungnam_345['date'])
chungnam_345['accident_hpn'] = chungnam_345['accident_num'].apply(lambda x: 1 if x > 0 else 0)
chungnam_345['accident_hpn'].value_counts()/chungnam_345['accident_hpn'].value_counts().sum()

0    0.724046
1    0.275954
Name: accident_hpn, dtype: float64

In [7]:
X = chungnam_345.drop(['date','accident_num','측정소명','accident_hpn'], axis = 1)
y = chungnam_345['accident_hpn']

In [8]:
print(X.shape)
print(y.shape)

(1127, 14)
(1127,)


In [9]:
X_trn, X_tes, y_trn, y_tes = train_test_split(X, y, test_size = 0.2, stratify = y, random_state = 42)

print(X_trn.shape)
print(X_tes.shape)
print(y_trn.shape)
print(y_tes.shape)

(901, 14)
(226, 14)
(901,)
(226,)


In [10]:
smote = SMOTENC(random_state=0, categorical_features = list(range(5,14)))
X_res, y_res = smote.fit_resample(X_trn,y_trn)

In [11]:
print(y_res.value_counts())
print(y_res.value_counts()/y_res.value_counts().sum())

1    652
0    652
Name: accident_hpn, dtype: int64
1    0.5
0    0.5
Name: accident_hpn, dtype: float64


In [12]:
cat_features = ['year', 'month','day','season','요일']
num_features = ['평균기온(°C)','일강수량(mm)','평균 풍속(m/s)','평균 상대습도(%)','평균 이슬점온도(°C)','O3','PM10','PM2.5']

enc = OneHotEncoder(handle_unknown = 'ignore')
scaler = MinMaxScaler()

enc.fit(X_res[cat_features])
scaler.fit(X_res[num_features])

train_oh = enc.transform(X_res[cat_features]).toarray()
test_oh = enc.transform(X_tes[cat_features]).toarray()

train_num = scaler.transform(X_res[num_features])
test_num = scaler.transform(X_tes[num_features])

print(train_oh.shape)
print(test_oh.shape)


(1304, 58)
(226, 58)


In [13]:
X_trn = np.concatenate([train_num,train_oh,X_res['workday'].values.reshape(-1,1)], axis = 1)
X_tes = np.concatenate([test_num,test_oh,X_tes['workday'].values.reshape(-1,1)], axis = 1)

print(X_trn.shape)
print(X_tes.shape)

(1304, 67)
(226, 67)


In [14]:
class TrnDataset(Dataset):

  def __init__(self,x_data,y_data):
    self.x_data = torch.FloatTensor(x_data)
    self.y_data = torch.LongTensor(y_data)
    self.len = len(self.x_data)

  def __getitem__(self,index):
    return self.x_data[index], self.y_data[index]

  def __len__(self):
    return self.len



In [15]:
class TesDataset(Dataset):

  def __init__(self,x_data,y_data):
    self.x_data = torch.FloatTensor(x_data)
    self.y_data = torch.LongTensor(y_data)
    self.len = len(self.x_data)

  def __getitem__(self,index):
    return self.x_data[index], self.y_data[index]

  def __len__(self):
    return self.len



In [16]:
train_data = TrnDataset(X_trn, y_res.values)
test_data = TesDataset(X_tes, y_tes.values)

train_loader = DataLoader(train_data,batch_size=100,shuffle=True)
test_loader = DataLoader(test_data,batch_size=100,shuffle=True)

In [17]:
arr, label = next(iter(train_loader))
print(arr.shape, label.shape)
arr2, label2 = next(iter(test_loader))
print(arr2.shape, label2.shape)

torch.Size([100, 67]) torch.Size([100])
torch.Size([100, 67]) torch.Size([100])


In [318]:
latent_size = 4

In [319]:
class Encoder(nn.Module):
  def __init__(self,latent_size):
    super(Encoder,self).__init__()

    self.encoder = nn.Sequential(
                      nn.Linear(67,64),
                      nn.ReLU(),
                      nn.Linear(64,32),
                      nn.ReLU(),
                      nn.Linear(32,16),
                      nn.ReLU(),
                      nn.Linear(16,latent_size),
                      nn.ReLU())

  def forward(self,x):
    return self.encoder(x)


class Decoder(nn.Module):
  def __init__(self, latent_size):
    super(Decoder,self).__init__()

    self.decoder = nn.Sequential(
                      nn.Linear(latent_size, 16),
                      nn.ReLU(),
                      nn.Linear(16,32),
                      nn.ReLU(),
                      nn.Linear(32,64),
                      nn.ReLU(),
                      nn.Linear(64,67),
                      nn.Sigmoid())

  def forward(self,x):
    return self.decoder(x)

In [320]:
encoder = Encoder(latent_size).to(device)
decoder = Decoder(latent_size).to(device)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(list(encoder.parameters())+list(decoder.parameters()), lr = 0.00005)

In [321]:
for ep in range(100):

  running_loss = 0.0

  for images, label in train_loader:
    inputs = images.to(device)
    optimizer.zero_grad()
    latent_var = encoder(inputs)
    outputs = decoder(latent_var)
    loss = criterion(inputs, outputs)
    loss.backward()
    optimizer.step()
    running_loss+=loss.item()
  cost = running_loss/len(train_loader)
  print('[%d]loss:%.3f'%(ep,cost))
  torch.save(encoder.state_dict(), './models/en_pth')
  torch.save(decoder.state_dict(), './models/de_pth')

[0]loss:0.233
[1]loss:0.232
[2]loss:0.231
[3]loss:0.230
[4]loss:0.229
[5]loss:0.228
[6]loss:0.227
[7]loss:0.226
[8]loss:0.226
[9]loss:0.225
[10]loss:0.223
[11]loss:0.222
[12]loss:0.221
[13]loss:0.219
[14]loss:0.218
[15]loss:0.217
[16]loss:0.214
[17]loss:0.212
[18]loss:0.210
[19]loss:0.208
[20]loss:0.205
[21]loss:0.202
[22]loss:0.198
[23]loss:0.194
[24]loss:0.189
[25]loss:0.183
[26]loss:0.177
[27]loss:0.170
[28]loss:0.161
[29]loss:0.153
[30]loss:0.143
[31]loss:0.133
[32]loss:0.123
[33]loss:0.113
[34]loss:0.104
[35]loss:0.096
[36]loss:0.090
[37]loss:0.086
[38]loss:0.082
[39]loss:0.079
[40]loss:0.078
[41]loss:0.076
[42]loss:0.075
[43]loss:0.074
[44]loss:0.073
[45]loss:0.073
[46]loss:0.072
[47]loss:0.072
[48]loss:0.072
[49]loss:0.071
[50]loss:0.071
[51]loss:0.071
[52]loss:0.071
[53]loss:0.071
[54]loss:0.071
[55]loss:0.070
[56]loss:0.071
[57]loss:0.071
[58]loss:0.070
[59]loss:0.070
[60]loss:0.071
[61]loss:0.070
[62]loss:0.070
[63]loss:0.070
[64]loss:0.070
[65]loss:0.070
[66]loss:0.070
[67]l

In [322]:
#forward training data
encoder.load_state_dict(torch.load('/content/models/en_pth'))
decoder.load_state_dict(torch.load('/content/models/de_pth'))

#latent vector
latent_features_trn = []
actual_trn = []

with torch.no_grad():
  for images, labels in train_loader:
    inputs = images.to(device)
    labels = labels.to(device)
    latent_var = encoder(inputs)

    latent_features_trn+=latent_var.cpu().tolist()
    actual_trn+=labels.cpu().tolist()

In [323]:
#forward test data
encoder.load_state_dict(torch.load('/content/models/en_pth'))
decoder.load_state_dict(torch.load('/content/models/de_pth'))

#latent vector
latent_features_tes = []
actual_tes = []

with torch.no_grad():
  for images, labels in test_loader:
    inputs = images.to(device)
    labels = labels.to(device)
    latent_var = encoder(inputs)

    latent_features_tes+=latent_var.cpu().tolist()
    actual_tes+=labels.cpu().tolist()

In [324]:
latent_features_trn = np.array(latent_features_trn)
latent_features_tes = np.array(latent_features_tes)
actual_tes = np.array(actual_tes)
actual_trn = np.array(actual_trn)

In [325]:
latent_features_trn=pd.DataFrame(latent_features_trn)
latent_features_tes=pd.DataFrame(latent_features_tes)
actual_tes = pd.DataFrame(actual_tes)
actual_trn = pd.DataFrame(actual_trn)

In [326]:
latent_features_trn.to_csv('/content/drive/MyDrive/지역 치안 안전 데이터 분석/data/Latent Variable/충남 cluster 345/latent_features_trn.csv')
latent_features_tes.to_csv('/content/drive/MyDrive/지역 치안 안전 데이터 분석/data/Latent Variable/충남 cluster 345/latent_features_tes.csv')
actual_tes.to_csv('/content/drive/MyDrive/지역 치안 안전 데이터 분석/data/Latent Variable/충남 cluster 345/actual_tes.csv')
actual_trn.to_csv('/content/drive/MyDrive/지역 치안 안전 데이터 분석/data/Latent Variable/충남 cluster 345/actual_trn.csv')

In [327]:
actual_trn = actual_trn.values.reshape(-1,)
actual_tes = actual_tes.values.reshape(-1,)

In [350]:
clf = RandomForestClassifier(n_estimators = 50, criterion = 'entropy',min_samples_split = 15, random_state = 0)
clf.fit(latent_features_trn, actual_trn)
y_trn_pred = clf.predict(latent_features_trn)

print(classification_report(actual_trn,y_trn_pred))

              precision    recall  f1-score   support

           0       0.89      0.90      0.89       652
           1       0.90      0.89      0.89       652

    accuracy                           0.89      1304
   macro avg       0.89      0.89      0.89      1304
weighted avg       0.89      0.89      0.89      1304



In [351]:
y_pred = clf.predict(latent_features_tes)
print(classification_report(actual_tes,y_pred))

              precision    recall  f1-score   support

           0       0.81      0.62      0.70       164
           1       0.38      0.61      0.47        62

    accuracy                           0.62       226
   macro avg       0.59      0.62      0.59       226
weighted avg       0.69      0.62      0.64       226

